In [1]:
import glob
import os
import json
import pandas as pd

In [22]:
# Import configs
with open("cr_config.json", "r") as jsonfile:
    config = json.load(jsonfile)

In [36]:
# Get samples with fastqs
fqs = glob.glob(config['work_dir'] + "/**/*R[12]*fastq.gz", recursive=True)

fq_samples = [i.split("/")[-2] for i in fqs]

# Get unique samples names
samples = []

for x in fq_samples:
    if x not in samples:
        samples.append(x)


mm_ret_M1_D37C_rep1
mm_ret_M3_whole
mm_ret_M18_whole
mm_ret_M1_SN_rep2
mm_ret_M3_CD73neg
mm_ret_M1_D8C_rep2
mm_ret_M24_whole
mm_ret_M1_SN_rep1
mm_ret_M18_CD73neg
mm_ret_M1_D8C_rep1
mm_ret_M12_CD73neg
mm_ret_M1_MeOH_rep2
mm_ret_M12_whole
mm_ret_M24_CD73neg
mm_ret_M1_D37C_rep2


In [24]:
# Make an export doc for each sample
for sample in samples:
    
    # bash doc for sbatch
    file_nm = "01_" + sample + ".sh"
    
    with open(path.join(work_dir, "src", file_nm), "w") as file:
        
        # Common lines for analysis
        shebang = "#!/bin/bash"
        usage = "# sbatch --cpus-per-task=32 --mem=120G --time=24:00:00 --gres=lscratch:100 " + file_nm
        md = "mkdir -p " + config["work_dir"] + "/cellranger_out"
        cd = "cd " + config["work_dir"] + "/cellranger_out"
        mod = "module load cellranger/" + config['cr_ver']
        line1 = "cellranger count \\"
        line2 = "--id " + sample + " \\"
        line3 = "--transcriptome " + config['cr_ref'] + " \\"
        line4 = "--fastqs \\"
        line5 = "-o kall_out/" + sample + " \\"
        line6 = "--h5ad \\"
        line7 = "--filter bustools \\"
        line8 = "--overwrite -t 16 \\"
        line9 = "--verbose \\"
        nl = ""  
        common_lines = [shebang, nl, usage, nl, mod, nl, 
                        line1, line2, line3, line4, line5, line6, line7, line8,
                        line9]
        
        # Get fastq files for the sample and join with directory
        tmp_fqs = df[df['sample'] == sample]['fastq'].tolist()
        tmp_fqs = [path.join(sample, i) for i in tmp_fqs]
        sample_fqs = ["%s \\" % i for i in tmp_fqs[:-1]]
        sample_fqs = sample_fqs + [tmp_fqs[-1]]
        
        # Join all lines and write to file
        lines = common_lines + sample_fqs
        file.writelines("%s\n" % l for l in lines)

/Volumes/data-1/scRNA-seq/06_200310/**/*R[12]*fastq.gz
